In [27]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [2]:
%%R
library(tidyverse)
# library(Seurat)
library(CellChat)


In [3]:
import scanpy as sc
import diopy
# diopy.output.write_h5(data_py, file = 'scdata.h5')

In [4]:
adata1=sc.read_h5ad('/home/maolp/data5/Gaofeng_All_matrix/Allcount/All_scanpyData/Data/Scanpy_merge_rmdoubulet_UMAP_label_lastCell.h5ad')

In [5]:
# diopy.output.write_h5(data_py, file = 'scdata.h5')
# in python
diopy.output.write_h5(adata1, file = '/home/maolp/data5/Gaofeng_All_matrix/Allcount/All_scanpyData/Data/scdata.h5')

In [6]:
import scanpy as sc
adata1=sc.read_h5ad("/home/maolp/data5/Gaofeng_All_matrix/Allcount/All_scanpyData/Data/Scanpy_merge_rmdoubulet_UMAP_label_lastCell.h5ad")

In [7]:
adata1.obs.to_csv("/home/maolp/Main_Gao_ScanpyProject20231130/Figure_plot/Figure2sup_Cellchat/Gao_Cellchat/CellChatData/adata1_umap_obsdata.csv")

In [8]:
%%R
# in R
# '/home/maolp/data5/Gaofeng_All_matrix/Allcount/All_scanpyData/Data/scdata.h5'
adata1 = dior::read_h5(file='/home/maolp/data5/Gaofeng_All_matrix/Allcount/All_scanpyData/Data/scdata.h5', target.object = 'seurat')

In [9]:
%%R
saveRDS(adata1,'/home/maolp/data5/Gaofeng_All_matrix/Allcount/All_scanpyData/Data/scdata.rds')

In [10]:
%%R
head(adata1@meta.data)

In [11]:
%%R
unique(adata1@meta.data$AdjustedID)

In [12]:
%%R
unique(adata1@meta.data$AdjustedIDoldweek)

In [13]:
%%R

library(dplyr)
library(stringr)


adata1@meta.data <- adata1@meta.data %>% mutate(
  Category = case_when(
    # Identify the initial letter
    str_detect(AdjustedID, "^B") ~ "PBMC",
    str_detect(AdjustedID, "^L") ~ "Liver",
    str_detect(AdjustedID, "^T") ~ "Thymus",
    str_detect(AdjustedID, "^S") ~ "Spleen",
    TRUE ~ "Other"  # for any other cases
  ),
  NumericPart = as.numeric(str_extract(AdjustedID, "\\d+\\.\\d+")),  # extract the numeric part
  Stage = ifelse(NumericPart <= 26, "Early", "Late")  # classify as "Early" or "Late"
)

In [14]:
%%R
adata1@meta.data$GroupStat<-paste0(adata1@meta.data$Category,"_",adata1@meta.data$Stage)

In [15]:
%%R
unique(adata1$GroupStat)
# unique(adata1$Cell_lineage)
# subadata1<-subset(adata1,Cell_lineage !="OTHERS")
subadata1<-adata1

In [16]:
%%R
process_cellchat233 <- function(adata,  id_value) {
  subset_data <- subset(adata, GroupStat== id_value)
  data.pbmc <- subset_data@assays$RNA@data
  # Use cell_type_label as the name of the column to use for group labels
  cellmeta <- data.frame(group = as.character(subset_data$Last_cell_type), row.names = names(subset_data$Last_cell_type))
#   print("test3")
  print(paste("Cellmeta rows: ", nrow(cellmeta)))
  print(paste("Subset data cells: ", length(subset_data@meta.data$Last_cell_type)))
  cellchat <- createCellChat(object = data.pbmc)
  cellchat <- addMeta(cellchat, meta = cellmeta, meta.name = "labels")
  cellchat <- setIdent(cellchat, ident.use = "labels")
  
  CellChatDB <- CellChatDB.human
  CellChatDB.use <- subsetDB(CellChatDB, search = "Cell-Cell Contact", key = "annotation")
  cellchat@DB <- CellChatDB.use
  
  cellchat <- subsetData(cellchat)
  cellchat <- identifyOverExpressedGenes(cellchat)
  cellchat <- identifyOverExpressedInteractions(cellchat)
  cellchat <- projectData(cellchat, PPI.human)

  cellchat <- computeCommunProb(cellchat)
  cellchat <- computeCommunProbPathway(cellchat)
  cellchat <- aggregateNet(cellchat)
  
  return(cellchat)
}

# Get a list of all unique AdjustedID values
# # adjusted_ids <- unique(adata1$AdjustedID)
# GroupStat_ids <- c( "PBMC_Early","PBMC_Late"  )
# PBMC_Early=process_cellchat233(subadata1,"PBMC_Early")


In [17]:

# ls -lh


In [18]:
%%R
adjusted_ids <- levels(adata1$AdjustedID)

adjusted_ids

In [19]:
mkdir -p /home/maolp/Allcount/All_scanpyData/Cellchat

In [20]:
%%R
GroupStat_ids <- c( "PBMC_Early","PBMC_Late"  )
# PBMC_late=process_cellchat233(subadata1,"PBMC_Late")

# results <- lapply(adjusted_ids, function(id) process_cellchat233(adata1, id))
GroupStat_res=lapply(GroupStat_ids , function(id) process_cellchat233(subadata1,id))
saveRDS(GroupStat_res, file = "/home/maolp/Allcount/All_scanpyData/Cellchat/PBMC_cellchat.rds")

In [ ]:
%%R
GroupStat_res=readRDS( "/home/maolp/Allcount/All_scanpyData/Cellchat/PBMC_cellchat.rds")

In [ ]:
%%R
# GroupStat_ids

In [ ]:
%%R
GroupStat_res2<-list(GroupStat_res[[2]],GroupStat_res[[1]])
PBMC_cellchat2 <- mergeCellChat(GroupStat_res2, add.names = c( "PBMC_Late" ,"PBMC_Early" ))

In [ ]:
%%R -w 3000 -h 3000 -r 300

netVisual_diffInteraction(PBMC_cellchat2  , weight.scale = T, measure = "weight")
# N1+N2

In [ ]:
%%R -w 3000 -h 3000 -r 300
# netVisual_diffInteraction(PBMC_cellchat  , weight.scale = T)
N1=netVisual_diffInteraction(PBMC_cellchat2  , weight.scale = T)

In [ ]:
%%R
PBMC_cellchat <- mergeCellChat(GroupStat_res, add.names =c( "PBMC_Early","PBMC_Late"  ))


In [ ]:
%%R -w 3000 -h 3000 -r 300

N2=netVisual_diffInteraction(PBMC_cellchat  , weight.scale = T, measure = "weight")
# N1+N2

In [ ]:
%%R -w 3000 -h 3000 -r 300
# netVisual_diffInteraction(PBMC_cellchat  , weight.scale = T)
N1=netVisual_diffInteraction(PBMC_cellchat2  , weight.scale = T)

In [ ]:
mkdir -p /home/maolp/Allcount/All_scanpyData/Cellchat

In [ ]:
%%R
str(PBMC_cellchat2@net)
# levels(PBMC_cellchat2@idents$PBMC_Late)

In [ ]:
%%R


In [ ]:
%%R
netVisual_heatmap666<-function (object, comparison = c(1, 2), measure = c("count", 
    "weight"), signaling = NULL, slot.name = c("netP", "net"), 
    color.use = NULL, color.heatmap = c("#2166ac", "#b2182b"), 
    title.name = NULL, width = NULL, height = NULL, font.size = 8, 
    font.size.title = 10, cluster.rows = FALSE, cluster.cols = FALSE, 
    sources.use = NULL, targets.use = NULL, remove.isolate = FALSE, 
    row.show = NULL, col.show = NULL) 
{
    if (!is.null(measure)) {
        measure <- match.arg(measure)
    }
    slot.name <- match.arg(slot.name)
    if (is.list(object@net[[1]])) {
        message("Do heatmap based on a merged object \n")
        obj1 <- object@net[[comparison[1]]][[measure]][cell_types_list ,cell_types_list ]
        obj2 <- object@net[[comparison[2]]][[measure]][cell_types_list ,cell_types_list ]
        net.diff <- obj2 - obj1
        if (measure == "count") {
            if (is.null(title.name)) {
                title.name = "Differential number of interactions"
            }
        }
        else if (measure == "weight") {
            if (is.null(title.name)) {
                title.name = "Differential interaction strength"
            }
        }
        legend.name = "Relative values"
    }
    else {
        message("Do heatmap based on a single object \n")
        if (!is.null(signaling)) {
            net.diff <- slot(object, slot.name)$prob[, , signaling]
            if (is.null(title.name)) {
                title.name = paste0(signaling, " signaling network")
            }
            legend.name <- "Communication Prob."
        }
        else if (!is.null(measure)) {
            net.diff <- object@net[[measure]]
            if (measure == "count") {
                if (is.null(title.name)) {
                  title.name = "Number of interactions"
                }
            }
            else if (measure == "weight") {
                if (is.null(title.name)) {
                  title.name = "Interaction strength"
                }
            }
            legend.name <- title.name
        }
    }
    net <- net.diff
    if ((!is.null(sources.use)) | (!is.null(targets.use))) {
        df.net <- reshape2::melt(net, value.name = "value")
        colnames(df.net)[1:2] <- c("source", "target")
        if (!is.null(sources.use)) {
            if (is.numeric(sources.use)) {
                sources.use <- rownames(net.diff)[sources.use]
            }
            df.net <- subset(df.net, source %in% sources.use)
        }
        if (!is.null(targets.use)) {
            if (is.numeric(targets.use)) {
                targets.use <- rownames(net.diff)[targets.use]
            }
            df.net <- subset(df.net, target %in% targets.use)
        }
        cells.level <- rownames(net.diff)
        df.net$source <- factor(df.net$source, levels = cells.level)
        df.net$target <- factor(df.net$target, levels = cells.level)
        df.net$value[is.na(df.net$value)] <- 0
        net <- tapply(df.net[["value"]], list(df.net[["source"]], 
            df.net[["target"]]), sum)
    }
    net[is.na(net)] <- 0
    if (remove.isolate) {
        idx1 <- which(Matrix::rowSums(net) == 0)
        idx2 <- which(Matrix::colSums(net) == 0)
        idx <- intersect(idx1, idx2)
        if (length(idx) > 0) {
            net <- net[-idx, ]
            net <- net[, -idx]
        }
    }
    mat <- net
    if (is.null(color.use)) {
        color.use <- scPalette(ncol(mat))
    }
    names(color.use) <- colnames(mat)
    if (!is.null(row.show)) {
        mat <- mat[row.show, ]
    }
    if (!is.null(col.show)) {
        mat <- mat[, col.show]
        color.use <- color.use[col.show]
    }
    if (min(mat) < 0) {
        color.heatmap.use = colorRamp3(c(min(mat), 0, max(mat)), 
            c(color.heatmap[1], "#f7f7f7", color.heatmap[2]))
        colorbar.break <- c(round(min(mat, na.rm = T), digits = nchar(sub(".*\\.(0*).*", 
            "\\1", min(mat, na.rm = T))) + 1), 0, round(max(mat, 
            na.rm = T), digits = nchar(sub(".*\\.(0*).*", "\\1", 
            max(mat, na.rm = T))) + 1))
    }
    else {
        if (length(color.heatmap) == 3) {
            color.heatmap.use = colorRamp3(c(0, min(mat), max(mat)), 
                color.heatmap)
        }
        else if (length(color.heatmap) == 2) {
            color.heatmap.use = colorRamp3(c(min(mat), max(mat)), 
                color.heatmap)
        }
        else if (length(color.heatmap) == 1) {
            color.heatmap.use = (grDevices::colorRampPalette((RColorBrewer::brewer.pal(n = 9, 
                name = color.heatmap))))(100)
        }
        colorbar.break <- c(round(min(mat, na.rm = T), digits = nchar(sub(".*\\.(0*).*", 
            "\\1", min(mat, na.rm = T))) + 1), round(max(mat, 
            na.rm = T), digits = nchar(sub(".*\\.(0*).*", "\\1", 
            max(mat, na.rm = T))) + 1))
    }
    df <- data.frame(group = colnames(mat))
    rownames(df) <- colnames(mat)
    col_annotation <- HeatmapAnnotation(df = df, col = list(group = color.use), 
        which = "column", show_legend = FALSE, show_annotation_name = FALSE, 
        simple_anno_size = grid::unit(0.2, "cm"))
    row_annotation <- HeatmapAnnotation(df = df, col = list(group = color.use), 
        which = "row", show_legend = FALSE, show_annotation_name = FALSE, 
        simple_anno_size = grid::unit(0.2, "cm"))
    ha1 = rowAnnotation(Strength = anno_barplot(rowSums(abs(mat)), 
        border = FALSE, gp = gpar(fill = color.use, col = color.use)), 
        show_annotation_name = FALSE)
    ha2 = HeatmapAnnotation(Strength = anno_barplot(colSums(abs(mat)), 
        border = FALSE, gp = gpar(fill = color.use, col = color.use)), 
        show_annotation_name = FALSE)
    if (sum(abs(mat) > 0) == 1) {
        color.heatmap.use = c("white", color.heatmap.use)
    }
    else {
        mat[mat == 0] <- NA
    }
    ht1 = Heatmap(mat, col = color.heatmap.use, na_col = "white", 
        name = legend.name, bottom_annotation = col_annotation, 
        left_annotation = row_annotation, top_annotation = ha2, 
        right_annotation = ha1, cluster_rows = cluster.rows, 
        cluster_columns = cluster.rows, row_names_side = "left", 
        row_names_rot = 0, row_names_gp = gpar(fontsize = font.size), 
        column_names_gp = gpar(fontsize = font.size), column_title = title.name, 
        column_title_gp = gpar(fontsize = font.size.title), column_names_rot = 90, 
        row_title = "Sources (Sender)", row_title_gp = gpar(fontsize = font.size.title), 
        row_title_rot = 90, heatmap_legend_param = list(title_gp = gpar(fontsize = 8, 
            fontface = "plain"), title_position = "leftcenter-rot", 
            border = NA, legend_height = unit(20, "mm"), labels_gp = gpar(fontsize = 8), 
            grid_width = unit(2, "mm")))
    return(list(ht1=ht1,mat = mat,  ha2= ha2,ha1=ha1,legend.name,row_annotation = row_annotation, color_info = color.heatmap.use, cluster_rows = cluster.rows))
}

# 如何将这个内容除了最后画图的那部分都返回来

In [ ]:
%%R
cell_types_list = c(
    "HSC_MPP",
    
    "MEP",
    "MEMP",
    "Pro-B",
    "Small pre-B",
    
    "CXCR5- Naïve B",
    "CXCR5+ Naïve B",

 
    "Treg",
    
    "Cycling Treg",
  
    "Naïve CD4 T",
    "Naïve CD8 T",
    
    
    'Tem',
        "Th17like_INNATE_T",
        'NK T',
        "Gamma Delta V1 T",
    "Gamma Delta V2 T",
    


     "ILC2/3",


    "CX3CR1+ NK",
    
    "CXCR6+ NK",
    "CD56highCD16low NK",

    
    "Myeloid-CD177",
    "Classical Monocytes",
    "CD14+PPBP+ Monocytes",
    "DC1",
    "DC2",
    "pDC",

    "Megakaryocytes",

    
    "Early_ERY",
    "Mid_ERY",
    "Late_ERY",

     "Others"


)


In [ ]:
%%R -w 6000 -h 3000 -r 500
library(gridExtra)
gg1 <- netVisual_heatmap666(PBMC_cellchat2,cluster.rows=F)$ht1
#> Do heatmap based on a merged object
gg2 <- netVisual_heatmap666(PBMC_cellchat2, measure = "weight")$ht1

# pdf(file = "/home/maolp/Allcount/All_scanpyData/Cellchat/gg3.pdf", width = 10, height = 5)

print(gg1+gg2)
# print(gg1)


# dev.off()

In [ ]:
%%R
setdiff(cell_types_list,colnames(PBMC_cellchat2@net[[c(1,2)[1]]][["count"]]))

In [ ]:
# %%R
# htone = Heatmap(run1$mat, col = run1$color_info, na_col = "white", 
#         name = run1$legend.name, bottom_annotation = run1$col_annotation, 
#         left_annotation = run1$row_annotation, top_annotation = run1$ha2, 
#         right_annotation = run1$ha1, cluster_rows = F, 
#         cluster_columns = F, row_names_side = "left", 
#         row_names_rot = 0, row_names_gp = gpar(fontsize = 8), 
#         column_names_gp = gpar(fontsize =8), column_title = NULL, 
#         column_title_gp = gpar(fontsize =8), column_names_rot = 90, 
#         row_title = "Sources (Sender)", row_title_gp = gpar(fontsize = 10), 
#         row_title_rot = 90, heatmap_legend_param = list(title_gp = gpar(fontsize = 8, 
#             fontface = "plain"), title_position = "leftcenter-rot", 
#             border = NA, legend_height = unit(20, "mm"), labels_gp = gpar(fontsize = 8), 
#             grid_width = unit(2, "mm")))
# htone

In [ ]:
%%R -w 6000 -h 3000 -r 500
library(gridExtra)
gg1 <- netVisual_heatmap(PBMC_cellchat2,cluster.rows=F)
#> Do heatmap based on a merged object
gg2 <- netVisual_heatmap(PBMC_cellchat2, measure = "weight")

# pdf(file = "/home/maolp/Allcount/All_scanpyData/Cellchat/gg3.pdf", width = 10, height = 5)

print(gg1+gg2)
# print(gg1)


# dev.off()

In [ ]:
%%R
# gg3

In [ ]:
%%R -w 2000 -h 1000 -r 300
gg1 <- compareInteractions(PBMC_cellchat, show.legend = F, group = c(1,2), color.use =c("#37BDA8","#525DAC"))
gg2 <- compareInteractions(PBMC_cellchat, show.legend = F, group = c(1,2), measure = "weight", color.use =c("#37BDA8","#525DAC"))
gg1 + gg2

In [ ]:
%%R
# netVisual_bubble

In [ ]:
%%R -w 5000 -h 3000 -r 300
netVisual_bubble(PBMC_cellchat2, sources.use = c(1:40) , targets.use =c(1),comparison = c(2,1), angle.x = 45,color.text =c("#37BDA8","#525DAC"))+coord_flip()
#> Comparing communications on a merged object

In [ ]:
# %%R -w 3000 -h 3000 -r 300
# object.list=GroupStat_res2
# num.link <- sapply(object.list, function(x) {rowSums(x@net$count) + colSums(x@net$count)-diag(x@net$count)})
# weight.MinMax <- c(min(num.link), max(num.link)) # control the dot size in the different datasets9
# gg <- list()
# for (i in 1:length(object.list)) {
#   gg[[i]] <- netAnalysis_signalingRole_scatter(object.list[[i]], title = names(object.list)[i], weight.MinMax = weight.MinMax)
# }
# #> Signaling role analysis on the aggregated cell-cell communication network from all signaling pathways
# #> Signaling role analysis on the aggregated cell-cell communication network from all signaling pathways
# patchwork::wrap_plots(plots = gg)

In [ ]:
%%R
object.list=GroupStat_res2
object.list[[1]]=netAnalysis_computeCentrality(object.list[[1]], slot.name = "netP") 
object.list[[2]]=netAnalysis_computeCentrality(object.list[[2]], slot.name = "netP") 
# object.list

names(object.list)<-c("PBMC_late","PBMC_Early")

In [ ]:
%%R -w  5000 -h 5000 -r 300
weight.max <- getMaxWeight(object.list, attribute = c("idents","count"))
par(mfrow = c(1,2), xpd=TRUE)
for (i in 1:length(object.list)) {
  netVisual_circle(object.list[[i]]@net$count, weight.scale = T, label.edge= F, edge.weight.max = weight.max[2], edge.width.max = 12, title.name = paste0("Number of interactions - ", names(object.list)[i]))
}

In [ ]:
%%R
# num.link

In [ ]:
%%R -w 3000 -h 3000 -r 300
num.link <- sapply(object.list, function(x) {rowSums(x@net$count) + colSums(x@net$count)-diag(x@net$count)})

weight.MinMax <- c(min(num.link), max(num.link)) 
# 控制不同数据集中的气泡大小
gg <- list()
for (i in 1:length(object.list)) {
  gg[[i]] <- netAnalysis_signalingRole_scatter(object.list[[i]], title = names(object.list)[i], weight.MinMax = weight.MinMax)
}

patchwork::wrap_plots(plots = gg)

In [ ]:
%%R -w 3000 -h 3000 -r 300
library(ComplexHeatmap)
i=1
pathway.union <- union(object.list[[i]]@netP$pathways, object.list[[i+1]]@netP$pathways)
ht1 = netAnalysis_signalingRole_heatmap(object.list[[i]], pattern = "outgoing", signaling = pathway.union, title = names(object.list)[i], width = 8, height = 8)
ht2 = netAnalysis_signalingRole_heatmap(object.list[[i+1]], pattern = "outgoing", signaling = pathway.union, title = names(object.list)[i+1], width = 8, height = 8)
draw(ht1 + ht2, ht_gap = unit(0.5, "cm"))

In [ ]:
# %%R -w 3000 -h 3000 -r 300
# pathway.union <- union(object.list[[i]]@netP$pathways, object.list[[i+1]]@netP$pathways)
# ht1 = netAnalysis_signalingRole_heatmap(object.list[[i]], pattern = "outgoing", signaling = pathway.union, title = names(object.list)[i], width = 8, height = 8)
# ht2 = netAnalysis_signalingRole_heatmap(object.list[[i+1]], pattern = "outgoing", signaling = pathway.union, title = names(object.list)[i+1], width = 8, height = 8)
# draw(ht1 + ht2, ht_gap = unit(0.5, "cm"))

In [ ]:
%%R -w 3000 -h 3000 -r 300
i=1
pathway.union <- union(object.list[[i]]@netP$pathways, object.list[[i+1]]@netP$pathways)
ht1 = netAnalysis_signalingRole_heatmap(object.list[[i]], pattern = "incoming", signaling = pathway.union, title = names(object.list)[i], width = 8, height = 8, color.heatmap = "GnBu")
ht2 = netAnalysis_signalingRole_heatmap(object.list[[i+1]], pattern = "incoming", signaling = pathway.union, title = names(object.list)[i+1], width = 8, height = 8, color.heatmap = "GnBu")
draw(ht1 + ht2, ht_gap = unit(0.5, "cm"))

In [ ]:
%%R
test1<-function (object, signaling = NULL, pattern = c("outgoing", "incoming",
"all"), slot.name = "netP", color.use = NULL, color.heatmap = "BuGn",
title = NULL, width = 10, height = 8, font.size = 8, font.size.title = 10,
cluster.rows = FALSE, cluster.cols = FALSE)
{
pattern <- match.arg(pattern)
if (length(slot(object, slot.name)$centr) == 0) {
stop("Please run netAnalysis_computeCentrality to compute the network centrality scores! ")
}
centr <- slot(object, slot.name)$centr
outgoing <- matrix(0, nrow = nlevels(object@idents), ncol = length(centr))
incoming <- matrix(0, nrow = nlevels(object@idents), ncol = length(centr))
dimnames(outgoing) <- list(levels(object@idents), names(centr))
dimnames(incoming) <- dimnames(outgoing)
for (i in 1:length(centr)) {
outgoing[, i] <- centr[[i]]$outdeg
incoming[, i] <- centr[[i]]$indeg
}
if (pattern == "outgoing") {
mat <- t(outgoing)
legend.name <- "Outgoing"
}
else if (pattern == "incoming") {
mat <- t(incoming)
legend.name <- "Incoming"
}
else if (pattern == "all") {
mat <- t(outgoing + incoming)
legend.name <- "Overall"
}
    return(mat)
}

In [ ]:
%%R
i=1
t1out=test1(object.list[[i]], pattern = "outgoing", signaling = pathway.union, title = names(object.list)[i], width = 8, height = 8, color.heatmap = "OrRd")
t1out

In [ ]:
%%R
t1in=test1(object.list[[i]], pattern = "incoming", signaling = pathway.union, title = names(object.list)[i], width = 8, height = 8, color.heatmap = "OrRd")
t1in

In [ ]:
%%R
t1=test1(object.list[[i]], pattern = "all", signaling = pathway.union, title = names(object.list)[i], width = 8, height = 8, color.heatmap = "OrRd")
t1
options(scipen = 999)
apply(t1, 1, sum)

pSum <- rowSums(t1)
pSum.original <- pSum
pSum <- -1/log(pSum)
pSum[is.na(pSum)] <- 0
pSum[is.na(pSum)] <- 0
    idx1 <- which(is.infinite(pSum) | pSum < 0)
    if (length(idx1) > 0) {
        values.assign <- seq(max(pSum) * 1.1, max(pSum) * 1.5, 
            length.out = length(idx1))
        position <- sort(pSum.original[idx1], index.return = TRUE)$ix
        pSum[idx1] <- values.assign[match(1:length(idx1), position)]
    }
pSum

In [ ]:
%%R
t2in=test1(object.list[[i+1]], pattern = "incoming", signaling = pathway.union, title = names(object.list)[i+1], width = 8, height = 8, color.heatmap = "OrRd")
t2in

In [ ]:
%%R
t2out=test1(object.list[[i+1]], pattern = "outgoing", signaling = pathway.union, title = names(object.list)[i+1], width = 8, height = 8, color.heatmap = "OrRd")
t2out

In [ ]:
%%R
t2=test1(object.list[[2]], pattern = "all", signaling = pathway.union, title = names(object.list)[i+1], width = 8, height = 8, color.heatmap = "OrRd")
t2
options(scipen = 999)
apply(t2, 1, sum)

In [ ]:
%%R -w 3000 -h 3000 -r 300
i=1
ht1 = netAnalysis_signalingRole_heatmap(object.list[[i]], pattern = "all", signaling = pathway.union, title = names(object.list)[i], width = 8, height = 8, color.heatmap = "OrRd")
ht2 = netAnalysis_signalingRole_heatmap(object.list[[i+1]], pattern = "all", signaling = pathway.union, title = names(object.list)[i+1], width = 8, height = 8, color.heatmap = "OrRd")
draw(ht1 + ht2, ht_gap = unit(0.5, "cm"))

In [ ]:
%%R -w 3000 -h 1200 -r 300
gg1 <- rankNet(PBMC_cellchat, mode = "comparison", stacked = T, do.stat = TRUE, color.use =c("#37BDA8","#525DAC"))
gg2 <- rankNet(PBMC_cellchat, mode = "comparison", stacked = F, do.stat = TRUE, color.use =c("#37BDA8","#525DAC"))
gg1 + gg2

In [ ]:
# %%R
# object <- rankNetPairwise(object)
# # test1=identifyEnrichedInteractions(PBMC_cellchat,from=c(1),to=c(2))

In [ ]:
%%R -w 6000 -h 3000 -r 300
gg1 <- netVisual_bubble(PBMC_cellchat, sources.use = 1, targets.use = c(1:20),  comparison = c(1, 2), max.dataset = 2, title.name = "Increased signaling in LS", angle.x = 45, remove.isolate = T)
#> Comparing communications on a merged object
gg2 <- netVisual_bubble(PBMC_cellchat, sources.use = 1, targets.use = c(1:20),  comparison = c(1, 2), max.dataset = 1, title.name = "Decreased signaling in LS", angle.x = 45, remove.isolate = T)
#> Comparing communications on a merged object
gg1 + gg2

In [ ]:
# %%R -w 6000 -h 3000 -r 300
# PBMC_cellchat <- computeNetSimilarityPairwise(PBMC_cellchat, type = "functional")
# #> Compute signaling network similarity for datasets 1 2
# PBMC_cellchat <- netEmbedding(PBMC_cellchat, type = "functional")
# #> Manifold learning of the signaling networks for datasets 1 2
# PBMC_cellchat <- netClustering(PBMC_cellchat, type = "functional")
# #> Classification learning of the signaling networks for datasets 1 2
# # Visualization in 2D-space
# netVisual_embeddingPairwise(PBMC_cellchat, type = "functional", label.size = 3.5)
# #> 2D visualization of signaling networks from datasets 1 2

In [ ]:
%%R
# str(PBMC_cellchat)

In [ ]:
%%R -w 6000 -h 3000 -r 300
# define a positive dataset, i.e., the dataset with positive fold change against the other dataset
pos.dataset = "PBMC_Early"
# define a char name used for storing the results of differential expression analysis
features.name = pos.dataset
# perform differential expression analysis
PBMC_cellchat <- identifyOverExpressedGenes(PBMC_cellchat, group.dataset = "datasets", pos.dataset = pos.dataset, 
                                            features.name = features.name, only.pos = FALSE, thresh.pc = 0.1, 
                                            thresh.fc = 0.1, thresh.p = 1)

In [ ]:
%%R -w 6000 -h 3000 -r 300
#> Use the joint cell labels from the merged PBMC_cellchat object
# # map the results of differential expression analysis onto the inferred cell-cell communications to easily manage/subset the ligand-receptor pairs of interest
net <- netMappingDEG(PBMC_cellchat, features.name = features.name)
# # extract the ligand-receptor pairs with upregulated ligands in LS
net.up <- subsetCommunication(PBMC_cellchat, net = net, datasets = "PBMC_Early",ligand.logFC = 0.2, receptor.logFC = NULL)
# extract the ligand-receptor pairs with upregulated ligands and upregulated recetptors in NL, i.e.,downregulated in LS
net.down <- subsetCommunication(PBMC_cellchat, net = net, datasets = "PBMC_Late",ligand.logFC = -0.1, receptor.logFC = -0.1)

gene.up <- extractGeneSubsetFromPair(net.up, PBMC_cellchat)
gene.down <- extractGeneSubsetFromPair(net.down, PBMC_cellchat)

pairLR.use.up = net.up[, "interaction_name", drop = F]
gg1 <- netVisual_bubble(PBMC_cellchat, pairLR.use = pairLR.use.up, sources.use = 1, targets.use = c(5:11), comparison = c(1, 2),  angle.x = 90, remove.isolate = T,title.name = paste0("Up-regulated signaling in ", names(object.list)[2]))
#> Comparing communications on a merged object
pairLR.use.down = net.down[, "interaction_name", drop = F]
gg2 <- netVisual_bubble(PBMC_cellchat, pairLR.use = pairLR.use.down, sources.use = 1, targets.use = c(5:11), comparison = c(1, 2),  angle.x = 90, remove.isolate = T,title.name = paste0("Down-regulated signaling in ", names(object.list)[2]))
#> Comparing communications on a merged object
gg1 + gg2

In [ ]:
# %%R -w 3000 -h 3000 -r 300
# # Chord diagram
# par(mfrow = c(1,2), xpd=TRUE)
# netVisual_chord_gene(object.list[[2]], sources.use = 4, targets.use = c(5:11), slot.name = 'net', net = net.up, lab.cex = 0.8, small.gap = 3.5, title.name = paste0("Up-regulated signaling in ", names(object.list)[2]))
# #> Note: The first link end is drawn out of sector 'MIF'.
# netVisual_chord_gene(object.list[[1]], sources.use = 4, targets.use = c(5:11), slot.name = 'net', net = net.down, lab.cex = 0.8, small.gap = 3.5, title.name = paste0("Down-regulated signaling in ", names(object.list)[2]))

In [ ]:
%%R -w 3000 -h 3000 -r 300
# netVisual_diffInteraction(PBMC_cellchat  , weight.scale = T)
N1=netVisual_diffInteraction(PBMC_cellchat  , weight.scale = T)

In [ ]:
%%R
GroupStat_res

In [ ]:
%%R -w 6000 -h 3000 -r 300
weight.max <- getMaxWeight(object.list, attribute = c("idents","count"))
par(mfrow = c(1,2), xpd=TRUE)
for (i in 1:length(object.list)) {
  netVisual_circle(object.list[[i]]@net$count, weight.scale = T, label.edge= F, edge.weight.max = weight.max[2], edge.width.max = 12, title.name = paste0("Number of interactions - ", names(object.list)[i]))
}

In [ ]:
%%R
# netVisual_bubble

In [ ]:
%%R -w 6000 -h 3000 -r 300
netVisual_bubble(PBMC_cellchat, sources.use = c(1:30), targets.use =1 ,  comparison = c(1, 2), angle.x = 45)

In [ ]:
%%R -w 6000 -h 3000 -r 300
netVisual_bubble(PBMC_cellchat, sources.use =  c(1:30), targets.use =2,  comparison = c(1, 2), angle.x = 45)
#> Comparing communications on a merged object

In [ ]:
%%R -w 6000 -h 3000 -r 300
netVisual_bubble(PBMC_cellchat, sources.use = 2, targets.use = c(1:30),  comparison = c(1, 2), angle.x = 45)
#> Comparing communications on a merged object

In [ ]:
%%R
colnames(PBMC_cellchat@net$PBMC_Early$prob)

In [ ]:
%%R -w 6000 -h 3000 -r 300
P1=netVisual_bubble(PBMC_cellchat, sources.use = c(1,4,7) , targets.use =c(1:39),  comparison = c(1, 2), angle.x = 45)
#> Comparing communications on a merged object
write.csv(P1$data,'P1data_cellchate.csv')

In [ ]:
%%R -w 6000 -h 3000 -r 300
P1=netVisual_bubble(PBMC_cellchat, sources.use = c(4,7,2) , targets.use =c(1:39),  comparison = c(1, 2), angle.x = 45)
P1
#> Comparing communications on a merged object
write.csv(P1$data,'/data1/maolp/Codeman/Project/Main_Gao_ScanpyProject20231003/Gao_plot/Figure2sup_Cellchat/Gao_Cellchat/CellChatData/P1data_cellchate_source.csv')
P1

In [ ]:
%%R -w 6000 -h 3000 -r 300
P2=netVisual_bubble(PBMC_cellchat, sources.use = c(1:39) , targets.use =c(4,7,2),  comparison = c(1, 2), angle.x = 45)
#> Comparing communications on a merged object
write.csv(P2$data,'/data1/maolp/Codeman/Project/Main_Gao_ScanpyProject20231003/Gao_plot/Figure2sup_Cellchat/Gao_Cellchat/CellChatData/P2data_cellchate.csv')
P2

In [ ]:
%%R -w 6000 -h 3000 -r 300
P2=netVisual_bubble(PBMC_cellchat, sources.use = c(1:39) , targets.use =c(1:30),  comparison = c(1, 2), angle.x = 45)
#> Comparing communications on a merged object
write.csv(P2$data,'P2data_cellchate.csv')

In [ ]:
%%R -w 6000 -h 3000 -r 300
netVisual_bubble(PBMC_cellchat, sources.use = c(1:30) , targets.use =3,  comparison = c(1, 2), angle.x = 45)
#> Comparing communications on a merged object

In [ ]:
%%R -w 6000 -h 3000 -r 300
netVisual_bubble(PBMC_cellchat, sources.use = 3, targets.use = c(1:30),  comparison = c(1, 2), angle.x = 45)
#> Comparing communications on a merged object

In [ ]:
%%R -w 6000 -h 3000 -r 300
# Chord diagram
par(mfrow = c(1,2), xpd=TRUE)
netVisual_chord_gene(object.list[[2]], sources.use = 4, targets.use = c(5:11), slot.name = 'net', net = net.up, lab.cex = 0.8, small.gap = 3.5, title.name = paste0("Up-regulated signaling in ", names(object.list)[2]))
#> Note: The first link end is drawn out of sector 'MIF'.
netVisual_chord_gene(object.list[[1]], sources.use = 4, targets.use = c(5:11), slot.name = 'net', net = net.down, lab.cex = 0.8, small.gap = 3.5, title.name = paste0("Down-regulated signaling in ", names(object.list)[2]))

In [ ]:
%%R -w 6000 -h 3000 -r 300
pathways.show <- c("CD99") 
weight.max <- getMaxWeight(object.list, slot.name = c("netP"), attribute = pathways.show) # control the edge weights across different datasets
par(mfrow = c(1,2), xpd=TRUE)
for (i in 1:length(object.list)) {
  netVisual_aggregate(object.list[[i]], signaling = pathways.show, layout = "circle", edge.weight.max = weight.max[1], edge.width.max = 10, signaling.name = paste(pathways.show, names(object.list)[i]))
}

In [ ]:
%%R -w 6000 -h 3000 -r 300
# cellchat@meta$datasets = factor(cellchat@meta$datasets, levels = c("NL", "LS")) # set factor level
plotGeneExpression(PBMC_cellchat, signaling = "CD99", split.by = "datasets", colors.ggplot = T)
#> The default behaviour of split.by has changed.
#> Separate violin plots are now plotted side-by-side.
#> To restore the old behaviour of a single split violin,
#> set split.plot = TRUE.
#>       
#> This message will be shown once per session.
#> Scale for 'y' is already present. Adding another scale for 'y', which will
#> replace the existing scale.
#> Scale for 'y' is already present. Adding another scale for 'y', which will
#> replace the existing scale.
#> Scale for 'y' is already present. Adding another scale for 'y', which will
#> replace the existing scale.

In [ ]:
# %%R
# process_cellchat <- function(adata,  id_value, cell_type_label) {
#   subset_data <- subset(adata, GroupStat== id_value)
#   data.pbmc <- subset_data@assays$RNA@data
  
#   # Use cell_type_label as the name of the column to use for group labels
#   cellmeta <- data.frame(group = as.character(subset_data[[cell_type_label]]), row.names = names(subset_data[[cell_type_label]]))
#   print("test3")
#   print(paste("Cellmeta rows: ", nrow(cellmeta)))
#   print(paste("Subset data cells: ", length(subset_data@meta.data[[cell_type_label]])))
#   cellchat <- createCellChat(object = data.pbmc)
#   cellchat <- addMeta(cellchat, meta = cellmeta, meta.name = "labels")
#   cellchat <- setIdent(cellchat, ident.use = "labels")
  
#   CellChatDB <- CellChatDB.human
#   CellChatDB.use <- subsetDB(CellChatDB, search = "Cell-Cell Contact", key = "annotation")
#   cellchat@DB <- CellChatDB.use
  
#   cellchat <- subsetData(cellchat)
#   cellchat <- identifyOverExpressedGenes(cellchat)
#   cellchat <- identifyOverExpressedInteractions(cellchat)
#   cellchat <- projectData(cellchat, PPI.human)

#   cellchat <- computeCommunProb(cellchat)
#   cellchat <- computeCommunProbPathway(cellchat)
#   cellchat <- aggregateNet(cellchat)
  
#   return(cellchat)
# }

# # Get a list of all unique AdjustedID values
# # adjusted_ids <- unique(adata1$AdjustedID)
# GroupStat_ids <- c( "PBMC_Early","PBMC_Late"  )
# # Run the function on all AdjustedID values and store the results in a list
# # Specify the column to use for cell type labels
# # results <- lapply(adjusted_ids, function(id) process_cellchat(adata1, id, "Last_cell_type"))
# # GroupStat_res=lapply(GroupStat_ids , function(id) process_cellchat(subadata1,id, "Last_cell_type"))
# # saveRDS(GroupStat_res, file = "GroupStat_res.rds")
# # Save the results as an RDS file
# # saveRDS(results, file = "cellchat_results.rds")

In [ ]:
%%R
# Merge the CellChat objects in the results list
# merged_cellchat <- mergeCellChat(results, add.names = adjusted_ids)

# Now merged_cellchat is a merged CellChat object

In [ ]:
# %%R
# process_cellchat2 <- function(adata, adjusted_id) {
#   subset_data <- subset(adata, AdjustedID == adjusted_id)
#   data.pbmc <- subset_data@assays$RNA@data
#   cellmeta <- data.frame(group = as.character(subset_data$Cell_lineage), row.names = names(subset_data$Cell_lineage))
  
#   cellchat <- createCellChat(object = data.pbmc)
#   cellchat <- addMeta(cellchat, meta = cellmeta, meta.name = "labels")
#   cellchat <- setIdent(cellchat, ident.use = "labels")
  
#   CellChatDB <- CellChatDB.human
#   CellChatDB.use <- subsetDB(CellChatDB, search = "Cell-Cell Contact", key = "annotation")
#   cellchat@DB <- CellChatDB.use
  
#   cellchat <- subsetData(cellchat)
#   cellchat <- identifyOverExpressedGenes(cellchat)
#   cellchat <- identifyOverExpressedInteractions(cellchat)
#   cellchat <- projectData(cellchat, PPI.human)

#   cellchat <- computeCommunProb(cellchat)
#   cellchat <- computeCommunProbPathway(cellchat)
#   cellchat <- aggregateNet(cellchat)
  
#   return(cellchat)
# }

# # Get a list of all unique AdjustedID values
# adjusted_ids <- unique(adata1$AdjustedID)

# # Run the function on all AdjustedID values and store the results in a list
# results2 <- lapply(adjusted_ids, function(id) process_cellchat2(adata1, id))

# # Save the results as an RDS file
# saveRDS(results2, file = "/home/maolp/Allcount/All_scanpyData/Data/cellchat_results_celllieange.rds")

In [25]:
%%R
results2=readRDS("cellchat_results.rds")

In [1]:
%%R
# unique(adata1$AdjustedID)
# str(results2[[1]])
# 提取每个列表元素的第一个列名中 _ 前面的部分
result <- lapply(1:32, function(x) {
    col_name <- colnames(results2[[x]]@data)[1]
    strsplit(col_name, "_")[[1]][1]
})


result_vector <- unlist(result)


print(result_vector)

In [ ]:
%%R
# str(adata1)

In [ ]:
%%R
# str(adata1)
library(dplyr)
unique_data <- adata1@meta.data %>%
  distinct(AdjustedID, Name)

unique_data
all(unique_data$Name==result_vector)


In [ ]:
%%R
# names(object.list2)<-unique_data$AdjustedID


In [ ]:
%%R
merged_cellchat2 <- mergeCellChat(results2, add.names =unique_data$AdjustedID)

In [ ]:
%%R
grep("B",unique_data$AdjustedID)

In [ ]:
%%R
adjusted_ids <-unique_data$AdjustedID
# Merge the CellChat objects in the results list
# merged_cellchat2 <- mergeCellChat(results2, add.names = adjusted_ids)

# Now merged_cellchat is a merged CellChat object

In [ ]:
%%R
object.list2=results2
object.list2 <- lapply(object.list2, function(x) netAnalysis_computeCentrality(x, slot.name = "netP"))

In [ ]:
%%R -w 3000 -h 1000 -r 300
gg1 <- compareInteractions(merged_cellchat2 , show.legend = F, group = c(1:32))
gg2 <- compareInteractions(merged_cellchat2 , show.legend = F, group = c(1:32), measure = "weight")
gg1 + gg2

In [ ]:
%%R
gg2$data

In [ ]:
%%R
library(tidyverse)

process_data <- function(data) {
  data <- data %>%
    separate(dataset, into = c("part1", "part2"), sep = "_") %>%
    mutate(
      Organ = substr(part1, 1, 1),
      Week = substr(part1, 2, 5)
    ) %>%
    arrange(Organ,Week) %>%
    mutate(
      Stage = ifelse(as.numeric(Week) > 26, "Late", "Early")
    )
  
  return(data)
}


data1 <- process_data(gg1$data)
data1$Organ_Stage <-paste0(data1$Organ,'_',data1$Stage)
data1_PBMC=data1[grep('B',data1$Organ),]
print(data1)


In [ ]:
%%R -w 1600 -h 1200 -r 300

library(ggplot2)
library(ggpubr)
library(ggsci)
data2 <- process_data(gg2$data)
data2$Organ_Stage <-paste0(data2$Organ,'_',data2$Stage)
data2$Organ_Stage<-factor(data2$Organ_Stage,level=c("B_Early", "B_Late", "L_Early","T_Early","S_Early"))
my_comparisons <- list(
  c("B_Early", "B_Late"),
  c("B_Early", "L_Early"),
  c("B_Early", "S_Early"),
  c("B_Early", "T_Early"),
  c("B_Late", "L_Early"),
  c("B_Late", "S_Early"),
  c("B_Late", "T_Early"),
  c("L_Early", "S_Early"),
  c("L_Early", "T_Early"),
  c("S_Early", "T_Early")
)
p <- ggplot(data2, aes(x = Organ_Stage, y = count)) +
#   geom_boxplot(aes())+
#   geom_violin(position = position_dodge(width = 1), scale = 'width') +
#   geom_boxplot(position = position_dodge(width = 1), outlier.size = 0.7, width = 0.1, show.legend = FALSE) 
geom_boxplot(alpha = .5) +
    geom_point(position = position_jitter(seed = 2023, width = .25),
               aes(color =Organ_Stage, shape =Organ_Stage ), alpha = .4, size = 3.5) +
    theme(panel.grid = element_blank(), 
          panel.background = element_blank(), 
          panel.border = element_blank(),
          axis.line = element_line(color = "black"),
          axis.text.x = element_text(size = 12, angle = 45, hjust = 1), 
          axis.text.y = element_text(size = 12),
          plot.title = element_text(hjust = 0.5),
          legend.title = element_blank(), 
          legend.key = element_blank()) +
    scale_fill_futurama() +
    scale_color_futurama()

p <- p + stat_compare_means(comparisons = my_comparisons, label = "p.signif", tip.length = 0.01) +
theme(panel.grid = element_blank(), 
        panel.background = element_blank(), 
        panel.border = element_blank(),
        axis.line = element_line(color = "black"),
        axis.text.x = element_text(size = 12, angle = 45, hjust = 1), 
        axis.text.y = element_text(size = 12),
        plot.title = element_text(hjust = 0.5),
        legend.title = element_blank(), 
        legend.key = element_blank()) +
  labs(x = '', y = '', title = 'Interaction strength')+scale_fill_manual(values = c("#FFA300","#F6313E", "#fb862b", "#0eb0c8", "#6a73cf"))


print(p)
ggsave(plot=p,"/home/maolp/Allcount/All_scanpyData/Cellchat/PBMC_cellchat_boxplot.pdf",width=4,height=4)

In [ ]:
%%R -w 1600 -h 1200 -r 300

library(ggplot2)
library(ggpubr)

data1$Organ_Stage<-factor(data1$Organ_Stage,level=c("B_Early", "B_Late", "L_Early","T_Early","S_Early"))
p2 <- ggplot(data1, aes(x = Organ_Stage, y = count)) +
geom_boxplot(alpha = .5) +
    geom_point(position = position_jitter(seed = 2023, width = .25),
               aes(color =Organ_Stage, shape =Organ_Stage ), alpha = .4, size = 3) +
    theme(panel.grid = element_blank(), 
          panel.background = element_blank(), 
          panel.border = element_blank(),
          axis.line = element_line(color = "black"),
          axis.text.x = element_text(size = 12, angle = 45, hjust = 1), 
          axis.text.y = element_text(size = 12),
          plot.title = element_text(hjust = 0.5),
          legend.title = element_blank(), 
          legend.key = element_blank()) +
    scale_fill_futurama() +
    scale_color_futurama()

my_comparisons <- list(
  c("B_Early", "B_Late"),
  c("B_Early", "L_Early"),
  c("B_Early", "S_Early"),
  c("B_Early", "T_Early"),
  c("B_Late", "L_Early"),
  c("B_Late", "S_Early"),
  c("B_Late", "T_Early"),
  c("L_Early", "S_Early"),
  c("L_Early", "T_Early"),
  c("S_Early", "T_Early")
)

p2 <- p2 + stat_compare_means(comparisons = my_comparisons, label = "p.signif", tip.length = 0.01) + theme_minimal() +
  theme(panel.grid = element_blank(), 
        panel.background = element_blank(), 
        panel.border = element_blank(),
        axis.line = element_line(color = "black"),
        axis.text.x = element_text(size = 12, angle = 45, hjust = 1), 
        axis.text.y = element_text(size = 12),
        plot.title = element_text(hjust = 0.5),
        legend.title = element_blank(), 
        legend.key = element_blank()) +
  labs(x = '', y = '', title = 'Number of inferred interactions')+scale_fill_manual(values = c("#FFA300","#F6313E", "#fb862b", "#0eb0c8", "#6a73cf"))

print(p2)
ggsave(plot=p2,"/home/maolp/Allcount/All_scanpyData/Cellchat/PBMC_cellchat_boxplot_inferred_interactions.pdf",width=4,height=4)

In [ ]:
%%R
unique_data2<-unique_data
unique_data2$Seq=seq(1,32)
unique_data2$Organ = substring(unique_data2$AdjustedID, 1, 1)
unique_data2$Week = substring(unique_data2$AdjustedID, 2, 5)


unique_data2 <- unique_data2 %>% arrange(Organ, Week)

print(unique_data2)
unique_data2

In [ ]:
%%R -w 10000 -h 12000 -r 300

num.link <- sapply(object.list2, function(x) {rowSums(x@net$count) + colSums(x@net$count)-diag(x@net$count)})
num.link 
weight.MinMax <- c(0,643) 

gg <- list()
j=0
for (i in unique_data2$Seq) {
    j=j+1
  gg[[j]] <- netAnalysis_signalingRole_scatter(object.list2[[i]], title = names(object.list2)[i], weight.MinMax = weight.MinMax)
}

patchwork::wrap_plots(plots = gg,ncol=4)

In [ ]:
%%R -w 10000 -h 12000 -r 300

num.link <- sapply(object.list2, function(x) {rowSums(x@net$count) + colSums(x@net$count)-diag(x@net$count)})
num.link 
weight.MinMax <- c(0,643) 
# # 控制不同数据集中的气泡大小
gg <- list()
for (i in 1:length(object.list2)) {
  gg[[i]] <- netAnalysis_signalingRole_scatter(object.list2[[i]], title = names(object.list2)[i], weight.MinMax = weight.MinMax)
}

patchwork::wrap_plots(plots = gg,ncol=3)

In [ ]:
# %%R
# str(merged_cellchat2 )

In [ ]:
# %%R
# # par(mfrow = c(1,2), xpd=TRUE)
# netVisual_diffInteraction(merged_cellchat2 , weight.scale = T)
# netVisual_diffInteraction(merged_cellchat2 , weight.scale = T, measure = "weight")

In [ ]:
# %%R -w 6000 -h 3000 -r 300
# gg1 <- netVisual_heatmap(merged_cellchat2)
# #> Do heatmap based on a merged object
# gg2 <- netVisual_heatmap(merged_cellchat2, measure = "weight")
# #> Do heatmap based on a merged object
# gg1 + gg2

In [ ]:
# %%R -w 6000 -h 3000 -r 300
# gg1 <- compareInteractions(merged_cellchat, show.legend = F, group = c(1,2))
# gg2 <- compareInteractions(merged_cellchat, show.legend = F, group = c(1,2), measure = "weight")
# gg1 + gg2

In [ ]:
# %%R -w 6000 -h 3000 -r 300
# gg1 <- gg1 + theme(axis.text.x = element_text(angle = 90, hjust = 1)) 
# gg2 <- gg2 + theme(axis.text.x = element_text(angle = 90, hjust = 1)) 

# gg1+gg2

In [ ]:
# %%R -w 3000 -h 1000 -r 300
# # str(gg1)
# # #  %>% arrange(dataset)
# # df_sorted <- gg1$data[order(gg1$data$dataset, decreasing = TRUE),]
# # df_sorted
# df=gg1$data
# # Get the first letter and the numeric part of the 'dataset' column
# first_letter <- substr(df$dataset, 1, 1)
# numeric_part <- as.numeric(gsub("\\D", "", df$dataset))

# # Sort df by the first letter in ascending order and the numeric part in ascending order
# df_sorted <- df[order(first_letter, numeric_part), ]
# df_sorted$Organ<-substring(df_sorted$dataset,1,1) 
# df_sorted$Organ<-factor(df_sorted$Organ,levels=c("B","L","T","S",labels=c("PBMC","Liver","Thymus","Spleen")))
# # Print the sorted data frame
# # print(df_sorted)
# df_sorted %>% ggplot(aes(as.character(dataset),count))+geom_col(aes(fill=Organ))+
# theme_bw()+theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
# facet_grid(. ~ Organ, space = "free", scales = "free", switch = "y") +xlab("")

In [ ]:
# %%R -w 3000 -h 1000 -r 300
# rank1<-rankNet(merged_cellchat2,mode="comparison",stacked=T,do.stat=T)
# rank2<-rankNet(merged_cellchat2,mode="comparison",stacked=F,do.stat=T)
# rank1+rank2

In [ ]:
# %%R
# levels(adjusted_ids)